<a href="https://colab.research.google.com/github/sahandv/scioscipred/blob/master/BERT_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT WORD EMBEDDING

Credits to https://towardsdatascience.com/nlp-extract-contextualized-word-embeddings-from-bert-keras-tf-67ef29f60a7b

In [1]:
!pwd
import sys
import os
print('env:',sys.executable)
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

/home/sahand/Projects/science_science
env: /home/sahand/anaconda3/envs/tf-1/bin/python


### Mount Google Drive
If on colab:

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!ls 'drive/My Drive/Data/1990-2018 corpus sentences abstract-title'

### Install requirements

In [ ]:
!rm -rf bert
!git clone https://github.com/google-research/bert

In [ ]:
!pip install tqdm

### Import requirements

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import time
import codecs
import collections
import json
import re

import pprint
import numpy as np
import tensorflow as tf

from BERT import modeling
from BERT import tokenization

import pandas as pd
from tqdm import tqdm

If on colab:

In [ ]:
from google.colab import files
sys.path.append('bert/')

### Make sure colab TPU is being used
If on colab:

In [ ]:
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

In [ ]:
from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

### Select which BERT model we want to use

In [3]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = '/home/sahand/Projects/science_science/BERT_weights/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

***** BERT pretrained directory: /home/sahand/Projects/science_science/BERT_weights/uncased_L-12_H-768_A-12 *****
/bin/bash: gsutil: command not found


### Define some global parameters

In [4]:
LAYERS = [-1,-2,-3,-4]
NUM_TPU_CORES = 272
MAX_SEQ_LENGTH = 500
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
BATCH_SIZE = 64

### Define wrapper classes for the input before processing and after processing

In [5]:
class InputExample(object):

  def __init__(self, unique_id, text_a, text_b=None):
    self.unique_id = unique_id
    self.text_a = text_a
    self.text_b = text_b
    
class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, unique_id, tokens, input_ids, input_mask, input_type_ids):
    self.unique_id = unique_id
    self.tokens = tokens
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.input_type_ids = input_type_ids

### Set up our Estimator

The estimator is a wrapper around the BertModel

In [6]:
def input_fn_builder(features, seq_length):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_unique_ids = []
  all_input_ids = []
  all_input_mask = []
  all_input_type_ids = []

  for feature in features:
    all_unique_ids.append(feature.unique_id)
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_input_type_ids.append(feature.input_type_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "unique_ids":
            tf.constant(all_unique_ids, shape=[num_examples], dtype=tf.int32),
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_type_ids":
            tf.constant(
                all_input_type_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
    })

    d = d.batch(batch_size=batch_size, drop_remainder=False)
    return d

  return input_fn
  
def model_fn_builder(bert_config, init_checkpoint, layer_indexes, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    unique_ids = features["unique_ids"]
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    input_type_ids = features["input_type_ids"]

    # The model is defined here
    model = modeling.BertModel(
        config=bert_config,
        is_training=False,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=input_type_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    if mode != tf.estimator.ModeKeys.PREDICT:
      raise ValueError("Only PREDICT modes are supported: %s" % (mode))

    tvars = tf.trainable_variables()
    scaffold_fn = None
    (assignment_map,
     initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(
         tvars, init_checkpoint)
    if use_tpu:

      def tpu_scaffold():
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
        return tf.train.Scaffold()

      scaffold_fn = tpu_scaffold
    else:
      tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    all_layers = model.get_all_encoder_layers()

    predictions = {
        "unique_id": unique_ids,
    }

    # Tredictions from the top 4 layers are extracted here
    for (i, layer_index) in enumerate(layer_indexes):
      predictions["layer_output_%d" % i] = all_layers[layer_index]

    output_spec = tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

### Transform InputExamples to InputFeatures

In [7]:
def convert_examples_to_features(examples, seq_length, tokenizer):
  """Loads a data file into a list of `InputBatch`s."""

  features = []
  for (ex_index, example) in enumerate(examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
      tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
      # Modifies `tokens_a` and `tokens_b` in place so that the total
      # length is less than the specified length.
      # Account for [CLS], [SEP], [SEP] with "- 3"
      _truncate_seq_pair(tokens_a, tokens_b, seq_length - 3)
    else:
      # Account for [CLS] and [SEP] with "- 2"
      if len(tokens_a) > seq_length - 2:
        tokens_a = tokens_a[0:(seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    input_type_ids = []
    tokens.append("[CLS]")
    input_type_ids.append(0)
    for token in tokens_a:
      tokens.append(token)
      input_type_ids.append(0)
    tokens.append("[SEP]")
    input_type_ids.append(0)

    if tokens_b:
      for token in tokens_b:
        tokens.append(token)
        input_type_ids.append(1)
      tokens.append("[SEP]")
      input_type_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < seq_length:
      input_ids.append(0)
      input_mask.append(0)
      input_type_ids.append(0)

    assert len(input_ids) == seq_length
    assert len(input_mask) == seq_length
    assert len(input_type_ids) == seq_length

    if ex_index < 5:
      tf.logging.info("*** Example ***")
      tf.logging.info("unique_id: %s" % (example.unique_id))
      tf.logging.info("tokens: %s" % " ".join(
          [tokenization.printable_text(x) for x in tokens]))
      tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
      tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
      tf.logging.info(
          "input_type_ids: %s" % " ".join([str(x) for x in input_type_ids]))

    features.append(
        InputFeatures(
            unique_id=example.unique_id,
            tokens=tokens,
            input_ids=input_ids,
            input_mask=input_mask,
            input_type_ids=input_type_ids))
  return features

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

### Converting a normal string sequence to InputExample

In [8]:
def read_sequence(input_sentences):
  examples = []
  unique_id = 0
  for sentence in input_sentences:
    line = tokenization.convert_to_unicode(sentence)
    examples.append(InputExample(unique_id=unique_id, text_a=line))
    unique_id += 1
  return examples

### Processing functions for inputs

A function which accepts *an array of strings* as a parameter and the desired *dimension (max 768) of the embedding output* and returns a dictionary with the token as key and the embedding vector as value.

The function builds the estimator and invokes a prediction based on the given inputs.

In [9]:
def get_features(input_text, dim=768,output_labels=None,output_file='embeddings.json',
             start_from_idx=0,use_TPU=False,doc_embedding=True):
    
    layer_indexes = LAYERS
    bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
    tokenizer = tokenization.FullTokenizer(
      vocab_file=VOCAB_FILE, do_lower_case=True)

    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
    run_config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      tpu_config=tf.contrib.tpu.TPUConfig(
          num_shards=NUM_TPU_CORES,
          per_host_input_for_training=is_per_host))

    examples = read_sequence(input_text)
    print("example len is ",len(examples))

    features = convert_examples_to_features(
      examples=examples, seq_length=MAX_SEQ_LENGTH, tokenizer=tokenizer)
    print('\nsample features are:',features[1].tokens)

    
    unique_id_to_feature = {}
    for feature in features:
        unique_id_to_feature[feature.unique_id] = feature

    print("features len is ",len(features))
    username = input("Press return to continue...")
    
    model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      layer_indexes=layer_indexes,
      use_tpu=use_TPU,
      use_one_hot_embeddings=True)

    # If TPU is not available, this will fall back to normal Estimator on CPU
    # or GPU.
    estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=use_TPU,
      model_fn=model_fn,
      config=run_config,
      predict_batch_size=BATCH_SIZE,
      train_batch_size=BATCH_SIZE)

    input_fn = input_fn_builder(
      features=features, seq_length=MAX_SEQ_LENGTH)

    # Get features
    output_dict = {}
    all_embeddings = []
    counter = 0
    for result in estimator.predict(input_fn, yield_single_examples=True):
        unique_id = int(result["unique_id"])
        feature = unique_id_to_feature[unique_id]
        output = collections.OrderedDict()
        sentence_dict = {}
        for (i, token) in enumerate(feature.tokens):
            layers = []
            for (j, layer_index) in enumerate(layer_indexes):
                layer_output = result["layer_output_%d" % j]
                layer_output_flat = np.array([x for x in layer_output[i:(i + 1)].flat])
                layers.append(layer_output_flat)
                
            if doc_embedding==True:
                if token == '[CLS]':
                    output[token] = list(sum(layers)[:dim]) # You can also concatenate or average layers here
                    sentence_dict.update({token:str(output[token])})
            else:
                output[token] = list(sum(layers)[:dim]) # You can also concatenate or average layers here
                sentence_dict.update({token:str(output[token])})
                
        output_dict[str(counter+start_from_idx)] = {'article':str(output_labels[counter]),'vectors':sentence_dict}
        counter += 1

    print('estimator is finalized and is at sentence', counter)
    print('witing to disk...')

    with open(output_file, 'w') as json_file:
        json.dump(output_dict, json_file)

    print('Finished writing embeddings with len',len(output_dict))

    return True

In [ ]:
def get_features(input_text, dim=768,output_labels=None,output_file='embeddings.json',
                 start_from_idx=0,use_TPU=False):
  layer_indexes = LAYERS

  bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

  tokenizer = tokenization.FullTokenizer(
      vocab_file=VOCAB_FILE, do_lower_case=True)

  is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
  run_config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      tpu_config=tf.contrib.tpu.TPUConfig(
          num_shards=NUM_TPU_CORES,
          per_host_input_for_training=is_per_host))

  examples = read_sequence(input_text)
  print("example len is ",len(examples))

  features = convert_examples_to_features(
      examples=examples, seq_length=MAX_SEQ_LENGTH, tokenizer=tokenizer)

  unique_id_to_feature = {}
  for feature in features:
    unique_id_to_feature[feature.unique_id] = feature

  print("features len is ",len(features))

  model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      layer_indexes=layer_indexes,
      use_tpu=use_TPU,
      use_one_hot_embeddings=True)

  # If TPU is not available, this will fall back to normal Estimator on CPU
  # or GPU.
  estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=use_TPU,
      model_fn=model_fn,
      config=run_config,
      predict_batch_size=BATCH_SIZE,
      train_batch_size=BATCH_SIZE)

  input_fn = input_fn_builder(
      features=features, seq_length=MAX_SEQ_LENGTH)

  # Get features
  output_dict = {}
  all_embeddings = []
  counter = 0
  for result in estimator.predict(input_fn, yield_single_examples=True):
    unique_id = int(result["unique_id"])
    feature = unique_id_to_feature[unique_id]
    output = collections.OrderedDict()
    sentence_dict = {}
    for (i, token) in enumerate(feature.tokens):
      layers = []
      for (j, layer_index) in enumerate(layer_indexes):
        layer_output = result["layer_output_%d" % j]
        layer_output_flat = np.array([x for x in layer_output[i:(i + 1)].flat])
        layers.append(layer_output_flat)
      output[token] = list(sum(layers)[:dim]) # You can also concatenate or average layers here 
      sentence_dict.update({token:str(output[token])})

    output_dict[str(counter+start_from_idx)] = {'article':str(output_labels[counter]),'vectors':sentence_dict}
    counter += 1
    
  print('estimator is finalized and is at sentence', counter)
  print('witing to disk...')

  with open(output_file, 'w') as json_file:
    json.dump(output_dict, json_file)

  print('Finished writing embeddings with len',len(output_dict))

  return True

### Run the model and get the embeddings

In [10]:
data = pd.read_csv('/mnt/16A4A9BCA4A99EAD/GoogleDrive/Data/Corpus/KPRIS/clean/abstract_title deflemm',names=['sentence'])
data

,sentence
0,lighting system disclosed comprising plurality...
1,photographic device eliminates photometric err...
2,print demand camera system camera unit incorpo...
3,simplified camera mechanism method allowing op...
4,beam splitter digital camera split light passe...
...,...
19705,display engine video graphic system includes p...
19706,facilitating interaction may enabled communica...
19707,present invention generally directed system me...
19708,disclosed system producing image including app...


In [11]:
tic = time.clock()

"""
period              start_from_idx
1990-2004           0
2005-2007           43482
2008-2010           17873+43482=61355
2011-2013           17873+43482+24613=85968
2014-2016           17873+43482+24613+26560=112528
2017-2018           17873+43482+24613+26560+40386=152914
"""
TPU_ADDRESS = ''
get_features(data['sentence'][:].values.tolist(),
  output_file = '/mnt/16A4A9BCA4A99EAD/tmp bert results/uncased a12 results.json',
  output_labels = list(data.index),
  start_from_idx=0)

# get_features(data['sentence'][70000:140000].values.tolist(), dim=300,
#   output_file = 'drive/My Drive/Data/1990-2018 corpus sentences abstract-title - vectors 300 - 70-140k.json',
#   output_labels = data['article_index'][70000:140000].values.tolist(),start_from_idx=70000)

# get_features(data['sentence'][140000:].values.tolist(), dim=300,
#   output_file = 'drive/My Drive/Data/1990-2018 corpus sentences abstract-title - vectors 300 - 140-205k.json',
#   output_labels = data['article_index'][140000:].values.tolist(),start_from_idx=140000)

toc = time.clock()
print('\nTime of process for all sentences:',toc-tic)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/home/sahand/anaconda3/envs/tf-1/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


example len is  19710
INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] lighting system disclosed comprising plurality image projection lighting device including first image projection lighting device central controller communication system permit central controller communicate plurality image projection lighting device first image projection lighting device comprised camera light valve wherein light valve used project first projected image onto projection surface camera first image projection light device capture least portion first projected image pr [SEP]
INFO:tensorflow:input_ids: 101 7497 2291 21362 9605 29018 3746 13996 7497 5080 2164 2034 3746 13996 7497 5080 2430 11486 4807 2291 9146 2430 11486 10639 29018 3746 13996 7497 5080 2034 3746 13996 7497 5080 11539 4950 2422 10764 16726 2422 10764 2109 2622 2034 11310 3746 3031 13996 3302 4950 2034 3746 13996 2422 5080 5425 2560 4664 2034 11310 3746 10975 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:input_ids: 101 6140 5157 4950 2291 4950 3131 13543 10014 3746 13617 3746 6364 4984 2854 4198 20010 6776 3085 15041 3131 2126 7473 12458 2401 8278 15041 3131 2089 13265 2449 4003 7451 3259 15110 3452 9812 7473 12458 2401 2828 3523 4003 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

INFO:tensorflow:  name = bert/encoder/layer_0/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:te

INFO:tensorflow:  name = bert/encoder/layer_5/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/

INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
I

/home/sahand/anaconda3/envs/tf-1/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [ ]:
files.download('drive/My Drive/Data/1990-2018 corpus sentences abstract-title - vectors - all')

In [ ]:
print(data.head(5))
print(data[-10:].values.tolist())

In [ ]:
!ls

Restart kernel here to free up space

In [1]:
import json
import pandas as pd
output_file = '/mnt/16A4A9BCA4A99EAD/tmp bert results/uncased a12 results.json'
with open(output_file) as f:
    results = json.loads(f.read())
results_clean = [results[x]['vectors']['[CLS]'][1:-1].split(', ') for x in results]
print(len(results_clean))
print(len(results_clean[0]))

19710
768


In [2]:
results_clean = pd.DataFrame(results_clean)
results_clean.to_csv('/mnt/16A4A9BCA4A99EAD/tmp bert results/uncased a12 results.csv',index=False)

Read the CSV

In [3]:
import pandas as pd
results = pd.read_csv('/mnt/16A4A9BCA4A99EAD/tmp bert results/uncased a12 results.csv')

In [4]:
results

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-1.026509,2.504253,0.503871,3.906834,-1.673985,2.106960,2.552151,1.821394,0.516371,-2.088084,...,-0.413400,2.032628,-0.006751,1.525444,-0.754306,-2.074172,-3.057952,1.717090,-0.970531,-0.126739
1,-1.911358,0.409554,1.801092,2.508711,0.555396,1.461679,1.617921,1.156700,0.860279,-1.815634,...,-1.458633,0.550587,1.575503,2.240396,-0.132238,-1.989019,-1.310395,2.471140,-1.049675,0.338686
2,-1.915477,1.071991,1.447053,3.872259,1.034379,2.023182,1.432221,0.486669,-0.324363,-0.316492,...,-0.827034,1.895121,-0.388996,1.900664,0.065443,-1.141070,-1.927792,1.595579,-0.142450,-0.374619
3,-1.431598,1.751323,1.036286,4.552248,-0.327865,-0.004574,1.287302,-0.500691,-1.073438,-1.240185,...,-2.950429,2.444352,1.486849,1.694134,-1.108196,-0.602699,-1.547576,3.101694,0.400036,-0.000953
4,-0.752660,1.733307,1.712471,1.633501,-0.840243,0.003783,2.205566,2.245139,0.219095,-1.854049,...,-0.821422,1.508105,0.029575,3.466185,-0.427217,-2.435753,-1.119000,0.712866,-1.061971,0.263015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19705,-2.902460,1.740900,2.163507,3.490622,0.939976,1.804377,2.560888,1.353295,-0.499162,-1.898771,...,-0.303553,1.572323,-0.434638,2.738880,-1.137716,-2.152367,-2.406308,1.453490,-0.187996,1.601338
19706,-2.356663,0.942000,2.887390,2.679152,0.134619,1.345732,0.769376,0.559078,0.101078,-1.326284,...,-1.175369,1.377228,0.937739,3.535929,-0.800291,-2.653674,-2.857182,2.988899,-0.237624,0.793541
19707,-2.637773,0.749630,2.165326,3.366657,-1.072513,1.090916,3.868383,0.457475,-0.581894,-2.917851,...,-0.215782,2.137603,0.429043,-0.074911,-0.784254,-2.052609,-1.813932,1.436829,0.376520,-0.170649
19708,-1.897598,0.158943,3.194553,3.641513,-0.887907,1.153879,2.913718,0.782875,1.262358,0.169749,...,-1.035436,2.242933,0.402399,2.037804,-0.124317,-2.031441,-3.417457,2.160957,0.867672,1.033926
